In [3]:

    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    import pickle
    from sklearn.metrics import confusion_matrix, accuracy_score,average_precision_score,classification_report,f1_score
    from sklearn.compose import ColumnTransformer
    import matplotlib.pyplot as plt
    from sklearn.ensemble import RandomForestClassifier
    import os

In [3]:
import os
from pathlib import Path
path = Path(os.getcwd())
print(path.parent)


C:\Users\deesaw\Desktop\GPFI\Flask app


In [25]:
    modelr = pickle.load(open('model_files/03randomforest_cleanEDI.sav', 'rb'))
    ohe_risk_catr = pickle.load(open('model_files/02ohe_cleanEDI.sav', 'rb'))
    le_enc_tgtr = pickle.load(open('model_files/01le_cleanEDI.sav', 'rb'))

C:\Users\deesaw\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
    X_test=pd.DataFrame()
    X_test['LIFNR']=['605000000000.00']
    X_test['MAT1']=['639660149.00']
    X_test['MAT2']=['9542DSBL500']
    X_test['MAT3']=['78731883230.00']
    X_test['SP_KUNNR']=['101485']
    X_test_risk_enc = pd.DataFrame(ohe_risk_catr.transform(X_test))
    #X=modelr.predict(X_test_risk_enc)
    
    

In [27]:
X_test_risk_enc.shape
X=modelr.predict(X_test_risk_enc)

(1, 5290)

In [30]:
le_enc_tgtr.inverse_transform(X)

array(['9542DL500B8'], dtype=object)

In [24]:
    print('Getting Trained')
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    import pickle
    from sklearn.metrics import confusion_matrix, accuracy_score,average_precision_score,classification_report,f1_score
    from sklearn.compose import ColumnTransformer
    import matplotlib.pyplot as plt
    from sklearn.ensemble import RandomForestClassifier
    import os

    filepath = os.path.join('dataset','Train.tsv')
    #filepath = os.path.join('dataset', filename)
    if os.path.isfile(filepath):
        print('udp', filepath)
        df_raw = pd.read_csv(filepath, sep='\t',dtype=object)
    else:
        print('abc')
        df_raw = pd.read_csv(r'dataset/Train.tsv', sep='\t',dtype=object)
    df = df_raw.copy()
    print(df.columns)
    df.columns=['LIFNR', 'MAT1', 'MAT2', 'MAT3', 'SP_KUNNR', 'MATNR']
    df=df[df.duplicated(subset=["LIFNR", "MAT1", "MAT2","MAT3","SP_KUNNR"],keep='last')]
    print(df.head())
    print(df.describe())
    print(df.isnull().sum())
    X1,y1=df[["LIFNR", "MAT1", "MAT2","MAT3","SP_KUNNR"]],df["MATNR"]
    #Label Encoding
    label_encoder_y = LabelEncoder()
    y= label_encoder_y.fit_transform(y1)
    file = open("models.pkl", "wb")
    pickle.dump(label_encoder_y, file)
    pickle.dump(label_encoder_y, open('model_files/01le_cleanEDI.pkl', 'wb'))

    #OnehotEncoding
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False,handle_unknown='ignore'), [0,1,2,3,4])], remainder='passthrough')
    X = np.array(ct.fit_transform(X1))
    pickle.dump(ct,open('model_files/02ohe_cleanEDI.sav', 'wb'))

    #Training Model
    classifier = RandomForestClassifier(n_estimators = 75, criterion = 'entropy', random_state = 0)
    classifier.fit(X, y)
    pickle.dump(classifier,  open('model_files/03randomforest_cleanEDI.sav', 'wb'))

    #Score
    y_pred = classifier.predict(X)
    y_prob = classifier.predict_proba(X)
    y_conf = [(np.max(pred_err)*100) for pred_err in y_prob]
    print(y_conf)
    cm = confusion_matrix(y, y_pred)
    #cm = confusion_matrix(y, classifier.predict(X))
    print( accuracy_score(y, y_pred))
    print(classification_report(y, y_pred))
    y_actual=label_encoder_y.inverse_transform(y)

Getting Trained
udp dataset\Train.tsv
Index(['LIFNR', 'MAT1', 'MAT2', 'MAT3', 'SP_KUNNR', 'MATNR'], dtype='object')
            LIFNR          MAT1      MAT2           MAT3 SP_KUNNR     MATNR
2  WMTD2C19350804  383081650.00  44223.00  4200044223.00   116485  44223/52
3  WMTD2C19350804  355519105.00  79292.00  3040079292.00   116485  79291/01
4  WMTD2C19350802  565137468.00  79262.00  3040079262.00   116485  79262/51
5  WMTD2C19350802  383081650.00  44223.00  4200044223.00   116485  44223/52
6  WMTD2C19350802  355519105.00  79292.00  3040079292.00   116485  79291/01
                  LIFNR          MAT1       MAT2       MAT3 SP_KUNNR     MATNR
count             50909         50909      50909      50909    50909     50909
unique              950          2702        720        611      307       795
top     605000000000.00  226900746.00  999999.00  999999.00   116485  22180/50
freq              14102          6717       7472       8003    17958      6717
LIFNR       0
MAT1        0
MAT2 

0.9802196075350135


C:\Users\deesaw\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.79      1.00      0.88        11
           1       1.00      0.50      0.67         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         9
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00       375
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         9
          16       1.00      1.00      1.00       146
          17       1.00    